In [90]:
import pandas as pd, numpy as np, datetime, math, calendar

In [ ]:
marin = pd.read_csv('../data/StandardizedData/Marin_Merged.csv')
team = pd.read_csv('../data/StandardizedData/Terrestrial_Vertebrate_Cols_Edited.csv')

In [3]:
#columns of the marin dataset
marin.columns

Index([u'Unnamed: 0', u'Project.ID', u'Deployment.ID', u'Image.ID',
       u'Location', u'Photo.Type', u'Photo.Type.Identified.by',
       u'Genus.Species', u'Uncertainty', u'IUCN.Identification.Number',
       u'Date_Time.Captured', u'Age', u'Sex', u'Individual.ID', u'Count',
       u'Animal.recognizable..Y.N.', u'individual.Animal.notes',
       u'Project.ID.x', u'Event', u'Array.Name', u'Deployment.Location.ID',
       u'Longitude.Resolution', u'Latitude.Resolution',
       u'Camera.Deployment.Begin.Date', u'Camera.Deployment.End.Date',
       u'Bait.Type', u'Bait.Description', u'Feature.Type',
       u'Feature.Type.methodology', u'Camera.ID', u'Quiet.Period.Setting',
       u'Restriction.on.access', u'Camera.Failure.Details', u'Project.ID.y',
       u'Make', u'Model', u'Serial.Number', u'Year.Purchased', u'Genus',
       u'Species'],
      dtype='object')

In [4]:
# Number of locations 

len(set(marin['Deployment.Location.ID']))

8

In [5]:
# Number of projects

len(set(marin['Project.ID']))

1

In [6]:
# SELECT the projections that we actually need
marin_simple = marin[['Project.ID',
'Deployment.Location.ID',
'Camera.Deployment.Begin.Date',
'Camera.Deployment.End.Date']]


marin_simple_per_location = marin_simple.groupby(['Project.ID', 'Deployment.Location.ID', 'Camera.Deployment.Begin.Date', 'Camera.Deployment.End.Date'])
marin_simple_per_location

In [7]:
marin_simple_per_location = marin_simple_per_location.count().reset_index()

In [8]:
marin_simple_per_location['Camera.Deployment.Begin.Date_datetime'] = marin_simple_per_location['Camera.Deployment.Begin.Date'].apply(lambda x: 
                                                                    datetime.datetime.strptime(x, '%Y-%m-%d'))

marin_simple_per_location['Camera.Deployment.End.Date_datetime'] = marin_simple_per_location['Camera.Deployment.End.Date'].apply(lambda x: 
                                                                    datetime.datetime.strptime(x, '%Y-%m-%d'))

marin_simple_per_location['trapnights'] = (marin_simple_per_location['Camera.Deployment.End.Date_datetime'] 
                                          - 
        marin_simple_per_location['Camera.Deployment.Begin.Date_datetime']).astype(pd.Timedelta).apply(lambda l: l.days)

In [9]:
marin_simple_per_location.head()

,Project.ID,Deployment.Location.ID,Camera.Deployment.Begin.Date,Camera.Deployment.End.Date,Camera.Deployment.Begin.Date_datetime,Camera.Deployment.End.Date_datetime,trapnights
0,ChedaJewel,WW63,2014-09-22,2014-10-17,2014-09-22,2014-10-17,25
1,ChedaJewel,WW63,2014-10-17,2014-12-17,2014-10-17,2014-12-17,61
2,ChedaJewel,WW63,2014-12-17,2015-02-04,2014-12-17,2015-02-04,49
3,ChedaJewel,WW63,2015-02-04,2015-04-07,2015-02-04,2015-04-07,62
4,ChedaJewel,WW63,2015-04-07,2015-06-02,2015-04-07,2015-06-02,56


In [47]:
#Generate a trap night count for each unique {ProjectId + DeploymentId + Month}

from dateutil.relativedelta import relativedelta

def getDaysInMonth(date):
    '''
    For a specific day d, it will return the number of days between d and the end of the month
    '''
    endDate = date + relativedelta(day=1, months=+1, days=-1)
    return (endDate - date).days + 1

def genCountByMonth(start, end):
    '''
    Given a date range, it will return a dictionary of tuples for (month, trapnight counts)
    '''
    totalcount = (end - start).days
    results = {}
    while totalcount > 0:
        daysInCurrentMonth = min(totalcount,getDaysInMonth(start))
        if(start.month in results.keys()):
            results[str(start.year)+'-'+str(start.month)] += daysInCurrentMonth
        else:
            results[str(start.year)+'-'+str(start.month)] = daysInCurrentMonth
        totalcount -= daysInCurrentMonth
        start += relativedelta(day=1, months=+1)
    return results

print genCountByMonth(marin_simple_per_location['Camera.Deployment.Begin.Date_datetime'][0], marin_simple_per_location['Camera.Deployment.End.Date_datetime'][0])

{'2014-9': 9, '2014-10': 16}


In [76]:
df = pd.DataFrame(columns=['Project.ID', 'Deployment.Location.ID', 'Month', 'Trapnights'])
y=0
for i in range(0,len(marin_simple_per_location)):
    projectId = marin_simple_per_location['Project.ID'][i]
    locationId = marin_simple_per_location['Deployment.Location.ID'][i]
    data = genCountByMonth(marin_simple_per_location['Camera.Deployment.Begin.Date_datetime'][i], marin_simple_per_location['Camera.Deployment.End.Date_datetime'][i])
    for month,trapnights in data.iteritems():
        df.loc[y] = [projectId, locationId, month, trapnights]
        y+=1

In [88]:
#final output -- trap night count for each unique {ProjectId + DeploymentId + Month}
df = df.groupby(['Project.ID', 'Deployment.Location.ID','Month']).sum().reset_index()
df

,Project.ID,Deployment.Location.ID,Month,Trapnights
0,ChedaJewel,WW63,2014-10,31.0
1,ChedaJewel,WW63,2014-11,30.0
2,ChedaJewel,WW63,2014-12,31.0
3,ChedaJewel,WW63,2014-9,9.0
4,ChedaJewel,WW63,2015-1,31.0
5,ChedaJewel,WW63,2015-10,31.0
6,ChedaJewel,WW63,2015-11,30.0
7,ChedaJewel,WW63,2015-12,31.0
8,ChedaJewel,WW63,2015-2,28.0
9,ChedaJewel,WW63,2015-3,31.0


In [89]:
df.to_csv('marin_dataset_trapnights.csv')